#Zadanie 4 (7 pkt)
Celem zadania jest zaimplementowanie algorytmu drzewa decyzyjnego ID3 dla zadania klasyfikacji. Trening i test należy przeprowadzić dla zbioru Iris. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania drzewa dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Implementacja funkcji entropii - **0.5 pkt**
* Implementacja funkcji entropii zbioru - **0.5 pkt**
* Implementacja funkcji information gain - **0.5 pkt**
* Zbudowanie poprawnie działającego drzewa klasyfikacyjnego i przetestowanie go na wspomnianym wcześniej zbiorze testowym. Jeśli w liściu występuje kilka różnych klas, decyzją jest klasa większościowa. Policzenie accuracy i wypisanie parami klasy rzeczywistej i predykcji. - **4 pkt**
* Przeprowadzenie eksperymentów dla różnych głębokości drzew i podziałów zbioru treningowego i testowego (zmiana wartości argumentu test_size oraz usunięcie random_state). W tym przypadku dla każdego eksperymentu należy wykonać kilka uruchomień programu i wypisać dla każdego uruchomienia accuracy. - **1.5 pkt**

In [53]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1)

In [54]:
# funkcja entropii
def entropy_func(class_count, num_samples:list):
    total_num_samples = sum(num_samples)
    entropy = sum([-num_samples[count]/total_num_samples* math.log(num_samples[count]/total_num_samples, 2) if count else 0 for count in range(class_count)])
    return entropy


# zbiór klas, jako parametr przyjmuje listę klas, zwraca jej długość i entropię zbioru
class Group:
    def __init__(self, group_classes:list):
        self.group_classes = group_classes
        self.entropy = self.group_entropy()
        self.size = len(group_classes) 

    def __len__(self):
        return len(self.group_classes)

 # oblicza entropię na podstawie liczby klas, liczby unikalnych klas i tego ile razy wystąpiły
    def group_entropy(self):
        unique_classes = list(np.unique(self.group_classes))
        class_count = len(unique_classes)
        num_samples = []
        for x in unique_classes:
            samples = np.count_nonzero(self.group_classes == x)
            num_samples.append(samples)
        return entropy_func(class_count, num_samples)

# węzeł, przyjmuje indeks atrybutu, wartość względem której następuje podział na lewą i prawą klasę, głębokość w drzewie, węzły dzieci (lewego i prawego) i wartość liścia
class Node:
    def __init__(self, split_feature, split_val, depth=None, child_node_a=None, child_node_b=None, val=None):
        self.split_feature = split_feature
        self.split_val = split_val
        self.depth = depth
        self.child_node_a = child_node_a
        self.child_node_b = child_node_b
        self.val = val

    # zwraca watość decyzyjną danego przykładu
    def predict(self, data):
        if self.val is None:
            if data[self.split_feature] <= self.split_val:
                return self.child_node_a.predict(data)
            else:
                return self.child_node_b.predict(data)
        else:
            return self.val
# drzewo decyzyjne, jako parametry przyjmuje maksymalną głębokość, tworzone na podstawie pierwszego węzła stanowiącego atrybut tree
class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
        self.tree = None

    # zwraca entropię każdego ze zbiorów
    @staticmethod
    def get_split_entropy(group_a, group_b):
        split_entropy_a = group_a.entropy
        split_entropy_b = group_b.entropy
        return split_entropy_a, split_entropy_b
         
    # informacja, którą zyskujemy dzięki klasyfikacji rodzica na dwójkę dzieci 
    def get_information_gain(self, parent_group, child_group_a, child_group_b):
        parent_entropy = parent_group.entropy
        child_a_entropy, child_b_entropy = self.get_split_entropy(child_group_a, child_group_b)
        information_gain = parent_entropy - child_a_entropy*(child_group_a.size/parent_group.size) -child_b_entropy*(child_group_b.size/parent_group.size)
        return information_gain

    # dzieli na lewą (a) i prawą (b) klasę względem jakieś wartości granicznej podziału, dzieli dane według tej wartości
    # tworzy obiekty rodzica 
    # zwraca uzyskaną informację oraz klasy lewego i prawego dziecka wraz danymi
    def get_left_right_split(self, data, feature_values, split_value, classes):
        a_classes = [classes[n] for n in range(len(classes)) if feature_values[n] <= split_value]
        b_classes = [classes[n] for n in range(len(classes)) if feature_values[n] > split_value]
        a_data = [data[n] for n in range(len(classes)) if feature_values[n] <= split_value]
        b_data = [data[n] for n in range(len(classes)) if feature_values[n] > split_value]
        parent_group = Group(classes)
        a_group = Group(a_classes)
        b_group = Group(b_classes)
        info_gain = self.get_information_gain(parent_group, a_group, b_group)
        return info_gain, a_classes, b_classes, a_data, b_data

    # najlepszy atrybut decyzyjny, zwraca uzyskaną informację, wartość atrybutu, klasy lewą i prawą wraz z danymi do nich należącymi
    def get_best_feature_split(self, data, feature_values, classes):
        best_info_gain = 0
        best_split_value = None
        best_a_classes = None
        best_b_classes = None
        best_a_data = None
        best_b_data = None
        for split_value in feature_values:
            info_gain, a_classes, b_classes, a_data, b_data = self.get_left_right_split(data, feature_values, split_value, classes)
            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_split_value = split_value
                best_a_classes = a_classes
                best_b_classes = b_classes
                best_a_data = a_data
                best_b_data = b_data
                return best_info_gain, best_split_value, best_a_classes, best_b_classes, best_a_data, best_b_data

    # najlepszy podział pod względem wszystkich atrybutów i ich wartości, zwraca zyskaną informację, najlepszy atrybut, jego wartość, klasy i dane lewej oraz prawej strony
    def get_best_split(self, data, classes):
        converted_data = [None]*len(data[0])
        for x in range(len(data[0])):
          feature_values = [data[i][x] for i in range(len(data))]
          converted_data[x] = feature_values
        best_info_gain = 0
        best_split_feature =  None
        best_split_value =  None
        best_a_classes = None
        best_b_classes = None
        best_a_data = None
        best_b_data = None
        feature_index = 0
        for feature_vals in converted_data:
            info_gain, split_value, a_classes, b_classes, a_data, b_data = self.get_best_feature_split(data, feature_vals, classes)
            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_split_value = split_value
                best_split_feature = feature_index
                best_a_classes = a_classes
                best_b_classes = b_classes
                best_a_data = a_data
                best_b_data = b_data
            feature_index += 1
        return best_info_gain, best_split_value, best_split_feature, best_a_classes, best_b_classes, best_a_data, best_b_data

# buduje drzewo decyzyjne, przyjmuje jako parametry dane, zbiór klas i głębokość, zwraca węzeł lub liść
    def build_tree(self, data, classes, depth=0):
        if depth == self.max_depth or len(set(classes)) == 1:
          samples = dict(Counter(classes))
          leaf_value = max(samples, key=samples.get) # jeśli w liściu występuje kilka klas, decyzją jest klasa większościowa 
          print("NEW LEAF")
          print("LEAF CLASS VALUE")
          print(leaf_value)
          print("\n")
          return Node(None, None, None, None, None, leaf_value)
        else:
            info_gain, split_value, split_feature, a_classes, b_classes, a_data, b_data = self.get_best_split(data, classes)
            if info_gain > 0:
                a_branch = self.build_tree(a_data, a_classes, depth + 1)
                b_branch = self.build_tree(b_data, b_classes, depth + 1)
            print("NEW DECISION NODE")
            print("INFORMATION GAIN")
            print(info_gain)
            print("DECISION PARAMETER INDEX")
            print(split_feature)
            print("LIMIT VALUE")
            print(split_value)
            print("\n")
            return Node(split_feature, split_value, depth, a_branch, b_branch)


    def predict(self, data):
        return self.tree.predict(data)


In [55]:
def get_accuracy(depth):
    dc = DecisionTreeClassifier(depth)
    dc.tree = dc.build_tree(x_train, y_train)

    predictions = []
    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        predictions.append(prediction)
    total_number = len(predictions)
    error = 0 
    for y_predicted, y in zip(predictions, y_test):
        if y_predicted != y:
            error += 1
    accuracy = (total_number-error)/total_number

    print("PARAMETERS")
    print(x_test)
    print("\n")
    print("PREDICTIONS" "|" "CLASS")
    for y_predicted, y in zip(predictions, y_test):
            print(y_predicted , "|", y)

    print("ACCURACY")
    print(accuracy)

In [15]:
# depth = 3
get_accuracy(3)

NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
0


NEW DECISION NODE
INFORMATION GAIN
0.2156524441211401
DECISION PARAMETER INDEX
3
LIMIT VALUE
0.2


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.3725087546767559
DECISION PARAMETER INDEX
1
LIMIT VALUE
3.0


NEW DECISION NODE
INFORMATION GAIN
0.23353054685632402
DECISION PARAMETER INDEX
0
LIMIT VALUE
5.5


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.09755145068629123
DECISION PARAMETER INDEX
3
LIMIT VALUE
1.9


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.02920935761790011
DECISION PARAMETER INDEX
3
LIMIT VALUE
2.2


NEW DECISION NODE
INFORMATION GAIN
0.16962312544795516
DECISION PARAMETER INDEX
0
LIMIT VALUE
6.5


PARAMETERS
[[6.3 2.5 4.9 1.5]
 [6.8 3.  5.5 2.1]
 [6.4 2.8 5.6 2.2]
 [5.6 3.  4.1 1.3]
 [4.9 3.6 1.4 0.1]
 [6.  3.  4.8 1.8]
 [6.3 2.3 4.4 1.3]
 [4.4 3.2 1.3 0.2]
 [4.4 2.9 1.4 0.2]
 [5.5 2.6 

Dla głębokości drzewa 3 domyślnego przykładu wartość accuracy wynosi ok. 66,7%. Nie jest to zły wynik, jednak najwyraźniej nie jest to optymalna głębokość. Należy więc ją zwiększyć w celu otrzymania większej dokładności.

In [16]:
# depth = 5
get_accuracy(5)

NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
1


NEW DECISION NODE
INFORMATION GAIN
0.33319765827861564
DECISION PARAMETER INDEX
0
LIMIT VALUE
4.8


NEW LEAF
LEAF CLASS VALUE
0


NEW DECISION NODE
INFORMATION GAIN
0.4166666666666666
DECISION PARAMETER INDEX
1
LIMIT VALUE
3.0


NEW DECISION NODE
INFORMATION GAIN
0.2156524441211401
DECISION PARAMETER INDEX
3
LIMIT VALUE
0.2


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.0009641372096677314
DECISION PARAMETER INDEX
2
LIMIT VALUE
5.1


NEW DECISION NODE
INFORMATION GAIN
0.4904528182865035
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.7


NEW LEAF
LEAF CLASS VALUE
2


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.01818090322224869
DECISION PARAMETER INDEX
2
LIMIT VALUE
5.6


NEW LEAF
LEAF CLASS VALUE
0


NEW DECISION NODE
INFORMATION GAIN
0.6979098932374422
DECISION PARAMETER INDEX
1
LIMIT VALUE
3.4


NE

Dla głębokości 5 accuracy jest już znacznie lepsze, bo wynosi ok. 93,3%. Sprawdzimy więc, czy zwiększenie głębokości poprawi dokładność na jeszcze lepszą.

In [17]:
# depth = 6
get_accuracy(6)

NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
1


NEW DECISION NODE
INFORMATION GAIN
0.13219280948873624
DECISION PARAMETER INDEX
0
LIMIT VALUE
5.0


NEW DECISION NODE
INFORMATION GAIN
0.33319765827861564
DECISION PARAMETER INDEX
0
LIMIT VALUE
4.8


NEW LEAF
LEAF CLASS VALUE
0


NEW DECISION NODE
INFORMATION GAIN
0.4166666666666666
DECISION PARAMETER INDEX
1
LIMIT VALUE
3.0


NEW DECISION NODE
INFORMATION GAIN
0.2156524441211401
DECISION PARAMETER INDEX
3
LIMIT VALUE
0.2


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.0020251228846716457
DECISION PARAMETER INDEX
0
LIMIT VALUE
6.1


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.0009641372096677314
DECISION PARAMETER INDEX
2
LIMIT VALUE
5.1


NEW DECISION NODE
INFORMATION GAIN
0.4904528182865035
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.7


NEW LEAF
LEAF CLASS VALUE
1



Dla głębokości 6 otrzymaliśmy tę samą wartość accuracy, co dla głębokości 6. Sprawdźmy wyniki dla mniejszej głębokości drzewa.

In [21]:
# depth = 1
get_accuracy(1)

NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.16962312544795516
DECISION PARAMETER INDEX
0
LIMIT VALUE
6.5


PARAMETERS
[[6.3 2.5 4.9 1.5]
 [6.8 3.  5.5 2.1]
 [6.4 2.8 5.6 2.2]
 [5.6 3.  4.1 1.3]
 [4.9 3.6 1.4 0.1]
 [6.  3.  4.8 1.8]
 [6.3 2.3 4.4 1.3]
 [4.4 3.2 1.3 0.2]
 [4.4 2.9 1.4 0.2]
 [5.5 2.6 4.4 1.2]
 [6.9 3.1 5.1 2.3]
 [5.5 4.2 1.4 0.2]
 [5.2 2.7 3.9 1.4]
 [6.5 3.  5.5 1.8]
 [7.7 3.  6.1 2.3]]


PREDICTIONS|CLASS
0 | 1
2 | 2
0 | 2
0 | 1
0 | 0
0 | 2
0 | 1
0 | 0
0 | 0
0 | 1
2 | 2
0 | 0
0 | 1
0 | 2
2 | 2
ACCURACY
0.4666666666666667


Zgodnie z przewidywaniami accuracy pogorszyła się do wartości 46,7%. Przetestujmy jeszcze głębokość 0.

In [22]:
# depth = 0
get_accuracy(0)

NEW LEAF
LEAF CLASS VALUE
0


PARAMETERS
[[6.3 2.5 4.9 1.5]
 [6.8 3.  5.5 2.1]
 [6.4 2.8 5.6 2.2]
 [5.6 3.  4.1 1.3]
 [4.9 3.6 1.4 0.1]
 [6.  3.  4.8 1.8]
 [6.3 2.3 4.4 1.3]
 [4.4 3.2 1.3 0.2]
 [4.4 2.9 1.4 0.2]
 [5.5 2.6 4.4 1.2]
 [6.9 3.1 5.1 2.3]
 [5.5 4.2 1.4 0.2]
 [5.2 2.7 3.9 1.4]
 [6.5 3.  5.5 1.8]
 [7.7 3.  6.1 2.3]]


PREDICTIONS|CLASS
0 | 1
0 | 2
0 | 2
0 | 1
0 | 0
0 | 2
0 | 1
0 | 0
0 | 0
0 | 1
0 | 2
0 | 0
0 | 1
0 | 2
0 | 2
ACCURACY
0.26666666666666666


Accuracy przyjmuje niezerową wartość ze względu na przypisywanie liściom wartości klasy większościowej. 

W kolejnych doświadczeniach usuwam parametr random state i zmieniam test_size na 0.25

In [26]:
# test_size = 0.25
# depth = 0 
get_accuracy(0)

NEW LEAF
LEAF CLASS VALUE
0


PARAMETERS
[[6.3 2.5 4.9 1.5]
 [6.1 2.8 4.  1.3]
 [6.2 3.4 5.4 2.3]
 [5.1 3.8 1.5 0.3]
 [6.7 2.5 5.8 1.8]
 [5.4 3.7 1.5 0.2]
 [4.9 3.  1.4 0.2]
 [5.5 2.6 4.4 1.2]
 [7.1 3.  5.9 2.1]
 [6.1 2.8 4.7 1.2]
 [5.6 2.7 4.2 1.3]
 [5.1 2.5 3.  1.1]
 [5.5 2.4 3.8 1.1]
 [6.4 2.9 4.3 1.3]
 [6.7 3.1 4.7 1.5]
 [6.3 2.8 5.1 1.5]
 [4.9 3.6 1.4 0.1]
 [4.7 3.2 1.3 0.2]
 [6.8 3.  5.5 2.1]
 [6.1 3.  4.9 1.8]
 [5.8 2.7 5.1 1.9]
 [4.7 3.2 1.6 0.2]
 [6.5 3.  5.5 1.8]
 [6.7 3.1 5.6 2.4]
 [5.7 4.4 1.5 0.4]
 [5.6 2.9 3.6 1.3]
 [6.2 2.2 4.5 1.5]
 [5.8 2.7 3.9 1.2]
 [5.8 2.7 5.1 1.9]
 [5.  3.  1.6 0.2]
 [5.7 3.  4.2 1.2]
 [7.7 3.  6.1 2.3]
 [5.6 3.  4.1 1.3]
 [4.4 3.  1.3 0.2]
 [4.8 3.4 1.6 0.2]
 [7.2 3.  5.8 1.6]
 [5.3 3.7 1.5 0.2]
 [6.3 2.3 4.4 1.3]]


PREDICTIONS|CLASS
0 | 1
0 | 1
0 | 2
0 | 0
0 | 2
0 | 0
0 | 0
0 | 1
0 | 2
0 | 1
0 | 1
0 | 1
0 | 1
0 | 1
0 | 1
0 | 2
0 | 0
0 | 0
0 | 2
0 | 2
0 | 2
0 | 0
0 | 2
0 | 2
0 | 0
0 | 1
0 | 1
0 | 1
0 | 2
0 | 0
0 | 1
0 | 2
0 | 1
0 | 0
0 | 0
0 | 2


In [27]:
# test_size = 0.25
# depth = 1
get_accuracy(1)

NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.5899413696066447
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.6


PARAMETERS
[[6.3 2.5 4.9 1.5]
 [6.1 2.8 4.  1.3]
 [6.2 3.4 5.4 2.3]
 [5.1 3.8 1.5 0.3]
 [6.7 2.5 5.8 1.8]
 [5.4 3.7 1.5 0.2]
 [4.9 3.  1.4 0.2]
 [5.5 2.6 4.4 1.2]
 [7.1 3.  5.9 2.1]
 [6.1 2.8 4.7 1.2]
 [5.6 2.7 4.2 1.3]
 [5.1 2.5 3.  1.1]
 [5.5 2.4 3.8 1.1]
 [6.4 2.9 4.3 1.3]
 [6.7 3.1 4.7 1.5]
 [6.3 2.8 5.1 1.5]
 [4.9 3.6 1.4 0.1]
 [4.7 3.2 1.3 0.2]
 [6.8 3.  5.5 2.1]
 [6.1 3.  4.9 1.8]
 [5.8 2.7 5.1 1.9]
 [4.7 3.2 1.6 0.2]
 [6.5 3.  5.5 1.8]
 [6.7 3.1 5.6 2.4]
 [5.7 4.4 1.5 0.4]
 [5.6 2.9 3.6 1.3]
 [6.2 2.2 4.5 1.5]
 [5.8 2.7 3.9 1.2]
 [5.8 2.7 5.1 1.9]
 [5.  3.  1.6 0.2]
 [5.7 3.  4.2 1.2]
 [7.7 3.  6.1 2.3]
 [5.6 3.  4.1 1.3]
 [4.4 3.  1.3 0.2]
 [4.8 3.4 1.6 0.2]
 [7.2 3.  5.8 1.6]
 [5.3 3.7 1.5 0.2]
 [6.3 2.3 4.4 1.3]]


PREDICTIONS|CLASS
2 | 1
0 | 1
2 | 2
0 | 0
2 | 2
0 | 0
0 | 0
0 | 1
2 | 2
2 | 1
0 | 1
0 | 1
0 | 1
0 | 1
2 |

In [28]:
# test_size = 0.25
# depth = 2 
get_accuracy(2)

NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
1


NEW DECISION NODE
INFORMATION GAIN
0.1725618534148577
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.1


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.05958939587466554
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.8


NEW DECISION NODE
INFORMATION GAIN
0.5899413696066447
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.6


PARAMETERS
[[6.3 2.5 4.9 1.5]
 [6.1 2.8 4.  1.3]
 [6.2 3.4 5.4 2.3]
 [5.1 3.8 1.5 0.3]
 [6.7 2.5 5.8 1.8]
 [5.4 3.7 1.5 0.2]
 [4.9 3.  1.4 0.2]
 [5.5 2.6 4.4 1.2]
 [7.1 3.  5.9 2.1]
 [6.1 2.8 4.7 1.2]
 [5.6 2.7 4.2 1.3]
 [5.1 2.5 3.  1.1]
 [5.5 2.4 3.8 1.1]
 [6.4 2.9 4.3 1.3]
 [6.7 3.1 4.7 1.5]
 [6.3 2.8 5.1 1.5]
 [4.9 3.6 1.4 0.1]
 [4.7 3.2 1.3 0.2]
 [6.8 3.  5.5 2.1]
 [6.1 3.  4.9 1.8]
 [5.8 2.7 5.1 1.9]
 [4.7 3.2 1.6 0.2]
 [6.5 3.  5.5 1.8]
 [6.7 3.1 5.6 2.4]
 [5.7 4.4 1.5 0.4]
 [5.6 2.9 3.6 1.3]
 [6.2 2.2 4.5 1.5]
 [5.8 2.7 3.9 1.2]
 [5.8 2.7 5.1 1.9]
 [5.  3.  1.6 0.2]
 

In [29]:
# test_size = 0.25
# depth = 3 
get_accuracy(3)

NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
0


NEW DECISION NODE
INFORMATION GAIN
0.3717090610018946
DECISION PARAMETER INDEX
1
LIMIT VALUE
2.7


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.19379270637390128
DECISION PARAMETER INDEX
3
LIMIT VALUE
1.5


NEW DECISION NODE
INFORMATION GAIN
0.1725618534148577
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.1


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
1


NEW DECISION NODE
INFORMATION GAIN
0.23067283487360113
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.7


NEW LEAF
LEAF CLASS VALUE
2


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.01097947063005457
DECISION PARAMETER INDEX
2
LIMIT VALUE
5.1


NEW DECISION NODE
INFORMATION GAIN
0.05958939587466554
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.8


NEW DECISION NODE
INFORMATION GAIN
0.5899413696066447
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.6


PARAMETERS
[[6.3 2.5 4.9 1.5]
 [6.1 2.8 4.  1.3]
 [6.2 3.4 5.4 

Dla rozmiaru testu 0.25 accuracy osiąga nestępujące wartości w zależności od głębokości:
głębokość 0: 28,9%
głębokość 1: 60,5%
głębokość 2: 81,6%
głębokość 3: 89,5%
Tutaj podobnie zauważamy tendencję rosnącą dokładności wraz ze zwiększaniem się głębokości drzewa.

In [35]:
# test_size = 0.05
# depth = 0
get_accuracy(0)

NEW LEAF
LEAF CLASS VALUE
2


PARAMETERS
[[6.7 3.3 5.7 2.1]
 [4.6 3.2 1.4 0.2]
 [5.4 3.9 1.3 0.4]
 [5.1 2.5 3.  1.1]
 [5.2 3.5 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.  3.2 1.2 0.2]
 [5.6 2.5 3.9 1.1]]


PREDICTIONS|CLASS
2 | 2
2 | 0
2 | 0
2 | 1
2 | 0
2 | 0
2 | 0
2 | 1
ACCURACY
0.125


In [36]:
# test_size = 0.05
# depth = 1
get_accuracy(1)

NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.3467668411590542
DECISION PARAMETER INDEX
3
LIMIT VALUE
1.8


PARAMETERS
[[6.7 3.3 5.7 2.1]
 [4.6 3.2 1.4 0.2]
 [5.4 3.9 1.3 0.4]
 [5.1 2.5 3.  1.1]
 [5.2 3.5 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.  3.2 1.2 0.2]
 [5.6 2.5 3.9 1.1]]


PREDICTIONS|CLASS
2 | 2
1 | 0
1 | 0
1 | 1
1 | 0
1 | 0
1 | 0
1 | 1
ACCURACY
0.375


In [37]:
# test_size = 0.05
# depth = 2
get_accuracy(2)

NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.3092477667284915
DECISION PARAMETER INDEX
3
LIMIT VALUE
1.5


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.3467668411590542
DECISION PARAMETER INDEX
3
LIMIT VALUE
1.8


PARAMETERS
[[6.7 3.3 5.7 2.1]
 [4.6 3.2 1.4 0.2]
 [5.4 3.9 1.3 0.4]
 [5.1 2.5 3.  1.1]
 [5.2 3.5 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.  3.2 1.2 0.2]
 [5.6 2.5 3.9 1.1]]


PREDICTIONS|CLASS
2 | 2
0 | 0
0 | 0
0 | 1
0 | 0
0 | 0
0 | 0
0 | 1
ACCURACY
0.75


In [38]:
# test_size = 0.05
# depth = 3
get_accuracy(3)

NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
1


NEW DECISION NODE
INFORMATION GAIN
0.3805904214244009
DECISION PARAMETER INDEX
3
LIMIT VALUE
0.3


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.08625244325283907
DECISION PARAMETER INDEX
3
LIMIT VALUE
1.7


NEW DECISION NODE
INFORMATION GAIN
0.3092477667284915
DECISION PARAMETER INDEX
3
LIMIT VALUE
1.5


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.3467668411590542
DECISION PARAMETER INDEX
3
LIMIT VALUE
1.8


PARAMETERS
[[6.7 3.3 5.7 2.1]
 [4.6 3.2 1.4 0.2]
 [5.4 3.9 1.3 0.4]
 [5.1 2.5 3.  1.1]
 [5.2 3.5 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.  3.2 1.2 0.2]
 [5.6 2.5 3.9 1.1]]


PREDICTIONS|CLASS
2 | 2
0 | 0
1 | 0
1 | 1
0 | 0
0 | 0
0 | 0
1 | 1
ACCURACY
0.875


Dla rozmiaru testu 0.05 accuracy osiąga nestępujące wartości w zależności od głębokości:
głębokość 0: 12,5%
głębokość 1: 37,5%
głębokość 2: 75%
głębokość 3: 87,5%
Dla mniejszego rozmaru testu otrzymujemy mniejszą dokładność. Wynika to z faktu,że mamy mniej danych i mniej klas.

In [51]:
# test_size = 0.5
# depth = 4
get_accuracy(4)

NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.12240441577942951
DECISION PARAMETER INDEX
3
LIMIT VALUE
1.8


NEW DECISION NODE
INFORMATION GAIN
0.5747406423382122
DECISION PARAMETER INDEX
2
LIMIT VALUE
1.9


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.3063694932371954
DECISION PARAMETER INDEX
2
LIMIT VALUE
5.3


NEW DECISION NODE
INFORMATION GAIN
0.21064415607264952
DECISION PARAMETER INDEX
3
LIMIT VALUE
0.2


PARAMETERS
[[5.  3.6 1.4 0.2]
 [6.2 2.2 4.5 1.5]
 [5.5 2.5 4.  1.3]
 [6.3 3.3 4.7 1.6]
 [5.6 3.  4.1 1.3]
 [6.9 3.1 5.1 2.3]
 [5.6 3.  4.5 1.5]
 [6.8 2.8 4.8 1.4]
 [5.4 3.4 1.7 0.2]
 [5.2 4.1 1.5 0.1]
 [4.9 3.6 1.4 0.1]
 [6.6 3.  4.4 1.4]
 [6.8 3.  5.5 2.1]
 [5.8 2.7 3.9 1.2]
 [5.1 2.5 3.  1.1]
 [6.7 3.1 4.7 1.5]
 [6.9 3.1 5.4 2.1]
 [5.1 3.7 1.5 0.4]
 [4.6 3.1 1.5 0.2]
 [5.  3.4 1.6 0.4]
 [7.1 3.  5.9 2.1]
 [6.3 2.3 4.4 1.3]
 [5.8 2.7 5.1 1.9]
 [6.

In [52]:
# test_size = 0.5
# depth = 6
get_accuracy(6)

NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW LEAF
LEAF CLASS VALUE
1


NEW DECISION NODE
INFORMATION GAIN
0.20881418925323647
DECISION PARAMETER INDEX
1
LIMIT VALUE
3.0


NEW DECISION NODE
INFORMATION GAIN
0.2333333333333334
DECISION PARAMETER INDEX
3
LIMIT VALUE
1.3


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.12240441577942951
DECISION PARAMETER INDEX
3
LIMIT VALUE
1.8


NEW DECISION NODE
INFORMATION GAIN
0.5747406423382122
DECISION PARAMETER INDEX
2
LIMIT VALUE
1.9


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.3063694932371954
DECISION PARAMETER INDEX
2
LIMIT VALUE
5.3


NEW DECISION NODE
INFORMATION GAIN
0.21064415607264952
DECISION PARAMETER INDEX
3
LIMIT VALUE
0.2


PARAMETERS
[[5.  3.6 1.4 0.2]
 [6.2 2.2 4.5 1.5]
 [5.5 2.5 4.  1.3]
 [6.3 3.3 4.7 1.6]
 [5.6 3.  4.1 1.3]
 [6.9 3.1 5.1 2.3]
 [5.6 3.  4.5 1.5]
 [6.8 2.8 4.8 1.4]
 [5.4 3.4 1.7 0.2]
 [5.2 4.1 1

Dla większych wartości test_size dokładność jest większa, jednak po przekroczeniu pewnej optymalnej głębokości, zwiększanie głębokości zmniejsza dokładność - dla rozmiaru testu 0.5, accuracy dla głębokości 4 wyniosła 98,7%, zaś już dla głębokości 6 - 86,7%.  Wynika to z "przetrenowania" - nadmiernego dopasowania do zbioru treningowego.

In [56]:
# test_size = 1
get_accuracy(7)

NEW LEAF
LEAF CLASS VALUE
0


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW LEAF
LEAF CLASS VALUE
1


NEW DECISION NODE
INFORMATION GAIN
0.5283208335737187
DECISION PARAMETER INDEX
0
LIMIT VALUE
6.0


NEW LEAF
LEAF CLASS VALUE
1


NEW DECISION NODE
INFORMATION GAIN
0.1768796874098555
DECISION PARAMETER INDEX
1
LIMIT VALUE
2.3


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
1


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.5283208335737187
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.9


NEW DECISION NODE
INFORMATION GAIN
0.12235574756214275
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.7


NEW DECISION NODE
INFORMATION GAIN
0.013482789361884295
DECISION PARAMETER INDEX
1
LIMIT VALUE
2.7


NEW DECISION NODE
INFORMATION GAIN
0.023278192847214496
DECISION PARAMETER INDEX
2
LIMIT VALUE
4.0


NEW LEAF
LEAF CLASS VALUE
2


NEW DECISION NODE
INFORMATION GAIN
0.1070643466599131
DECISION PARAMETER INDEX
2
LIMIT VALUE
5.1


NEW LEAF
LEAF CLASS VALUE
2


NE

**Wnioski**
Algorytm ID3 jest dobrym narzędziem klasyfikacji - dla głębokości 3 osiąga dokładność 80-90%.
Dla większej liczby danych możemy przeprowadzać przeszukiwanie na większych głębokościach.
Dla zbyt dużych głębokości występuje przetrenowanie.